In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd 'drive/My Drive/chatbot'

/content/drive/My Drive/chatbot


In [ ]:
#importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
#loading question and answer embeddings of train and validation data from disk
train_embeds=pd.read_csv('re_train_embeds.csv')
validation_embeds=pd.read_csv('re_validation_embeds.csv')

In [ ]:
#function to preprocess question and answer embeddings post loading from disk
def preprocess_embeds(embed):
  output=[]
  embed=embed[2:-2]
  embed=embed.replace('\n','')
  for i in embed.split(' '):
    if i!=' ' and i!='':
      output.append(i)
  return (output)


In [ ]:
#preprocessing embeddings post loading from disk for train data
from tqdm.notebook import tqdm
tqdm.pandas()
train_embeds['question_embeds']=train_embeds.short_question_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))
train_embeds['answer_embeds']=train_embeds.short_answer_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))

In [ ]:
#preprocessing embeddings post loading from validation data
validation_embeds['question_embeds']=validation_embeds.short_question_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))
validation_embeds['answer_embeds']=validation_embeds.short_answer_embed_numpy.progress_apply(lambda x: preprocess_embeds(x))

In [ ]:
#function to l2 normalize each embedding
def normalize(x):
  x=np.array(x,dtype=np.float64)
  return (x)/(np.linalg.norm(x))

In [ ]:
#normalizing train question and answer embeddings
train_embeds['question_embeds_norm']=train_embeds['question_embeds'].apply(lambda x: normalize(x))
train_embeds['answer_embeds_norm']=train_embeds['answer_embeds'].apply(lambda x: normalize(x))

In [ ]:
#normalizing validation question and answer embeddings
validation_embeds['question_embeds_norm']=validation_embeds['question_embeds'].apply(lambda x: normalize(x))
validation_embeds['answer_embeds_norm']=validation_embeds['answer_embeds'].apply(lambda x: normalize(x))

In [ ]:
#Cleaning the gpt train data
train_gpt_data=train_embeds[['short_question','short_answer','question_embeds_norm','answer_embeds_norm']].copy()
train_gpt_data.columns=['question','answer','Q_FFNN_embeds','A_FFNN_embeds']
train_gpt_data.head(3)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,can an antibiotic through an iv give you a ras...,yes it can even after you have finished the pr...,"[-0.0015947532801336905, 0.028313202930346674,...","[-0.033252108881325626, 0.036488885218617664, ..."
1,can you test positive from having the hep b va...,test positive for what if you had a hep b vacc...,"[0.02388940038582553, 0.03578648615547307, -0....","[0.00845440863602465, 0.0381747505749067, 0.01..."
2,what are the dietary restrictions for celiac d...,omitting gluten from the diet is the key to co...,"[0.01185235921221315, 0.03739971001082176, -0....","[0.0030481943703320473, 0.024438320236403705, ..."


In [ ]:
#saving gpt training data to disk
train_gpt_data.to_pickle("./train_gpt_data.pkl")

In [ ]:
#cleaning gpt validation data
validation_gpt_data=validation_embeds[['short_question','short_answer','question_embeds_norm','answer_embeds_norm']].copy()
validation_gpt_data.columns=['question','answer','Q_FFNN_embeds','A_FFNN_embeds']
validation_gpt_data.head(3)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,do i have a yeast infection,hi this can be a vaginal fungal infection whic...,"[-0.012255767552121263, 0.011278647301171585, ...","[-0.038504731105611124, 0.005560366067635298, ..."
1,i need to buy health insurance asap what do i ...,go to healthcare gov call my husband 407 222 9...,"[0.08883379130035676, 0.039120400705552394, -0...","[0.0843680653010217, 0.04661907941883923, 0.00..."
2,i had an acute ebv antibody test done and my r...,hi yes a chronic ebv infection also can be rul...,"[-0.016364305356089962, 0.001889975145291855, ...","[-0.022903376308870023, -0.0005906026628375108..."


In [ ]:
#saving the gpt validation data to disk
validation_gpt_data.to_pickle("./validation_gpt_data.pkl")

In [ ]:
#loading train and validation gpt data from disk
train_gpt_data=pd.read_pickle('./train_gpt_data.pkl')
validation_gpt_data=pd.read_pickle('./validation_gpt_data.pkl')

In [ ]:
train_gpt_data.shape,validation_gpt_data.shape

((23802, 4), (5950, 6))

In [ ]:
#displaying the train gpt data
train_gpt_data.head(5)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,can an antibiotic through an iv give you a ras...,yes it can even after you have finished the pr...,"[-0.0015947532801336905, 0.028313202930346674,...","[-0.033252108881325626, 0.036488885218617664, ..."
1,can you test positive from having the hep b va...,test positive for what if you had a hep b vacc...,"[0.02388940038582553, 0.03578648615547307, -0....","[0.00845440863602465, 0.0381747505749067, 0.01..."
2,what are the dietary restrictions for celiac d...,omitting gluten from the diet is the key to co...,"[0.01185235921221315, 0.03739971001082176, -0....","[0.0030481943703320473, 0.024438320236403705, ..."
3,i have had a pneumonia shot can i get either a...,you can always catch an illness from a child b...,"[0.019805493645469713, 0.030971378024292198, -...","[0.006673388339564631, 0.030375235844495984, 0..."
4,my baby ate her on poop my baby ate poop 4 day...,hi one of ours did that she is now 47 and a mu...,"[0.0023687095008356495, 0.035630223208101997, ...","[-0.022711190802570207, 0.03978915434444173, 0..."


**Getting similar question and preparing actual gpt input**

In [ ]:
#installing faiss and other required libraries
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

--2021-02-11 03:44:32--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.92.24, 104.17.93.24, 2606:4700::6811:5c18, ...
Connecting to anaconda.org (anaconda.org)|104.17.92.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20210211T034432Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=IQoJb3JpZ2luX2VjELb%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIQDqFqsp1qZpg7e7Z%2B7G8p0CvRzXycnO3CtjubwoBLENkQIgZvTK%2BzVvFmPdRWtZlD2EDln2q7oMA26vAsgA1nT%2Ft9wqvQMIr%2F%2F%2F%2F%2F%2F%2

In [ ]:

import os
import csv
from tqdm import tqdm
import argparse
from glob import glob
import faiss
from multiprocessing import Pool, cpu_count
from math import ceil
from collections import defaultdict

#creating faiss index for semantic search 
#https://github.com/ash3n/DocProduct/blob/master/docproduct/train_embedding_to_gpt2_data.py
qa = pd.read_pickle('./train_gpt_data.pkl')
question_bert = qa["Q_FFNN_embeds"].tolist()
answer_bert = qa["A_FFNN_embeds"].tolist()
question_bert = np.array(question_bert)
answer_bert = np.array(answer_bert)

question_bert = question_bert.astype('float32')
answer_bert = answer_bert.astype('float32')
answer_index = faiss.IndexFlatIP(answer_bert.shape[-1])
answer_index.add(answer_bert)




Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.


In [ ]:
#defining function to prepare the actual gpt training data by retrieving similar question and answer pairs
#https://github.com/ash3n/DocProduct/blob/master/docproduct/train_embedding_to_gpt2_data.py
def preparing_gpt_training_data(question,answer,question_embedding):
  topk=20
  scores,indices=answer_index.search(
                  np.array([question_embedding]).astype('float32'), topk)
  q_sub=qa.iloc[indices.reshape(20)]
  
  line = '`QUESTION: %s `ANSWER: %s' % (
                        question, answer)
  encoded_len=len(tokenizer.encode(line))
  for i in q_sub.iterrows():
    line='`QUESTION: %s `ANSWER: %s ' % (i[1]['question'],i[1]['answer']) + line
    line=line.replace('\n','')
    encoded_len=len(tokenizer.encode(line))
    if encoded_len>=1024:
      break
  return tokenizer.encode(line)[-1024:]

In [ ]:
#preparing the gpt data for the training data
qa['gpt_data']=qa.progress_apply(lambda x: preparing_gpt_training_data(x.question,x.answer,x.Q_FFNN_embeds),axis=1)

In [ ]:
#preparing the gpt data for the validation data
validation_gpt_data['gpt_data']=validation_gpt_data.progress_apply(lambda x: preparing_gpt_training_data(x.question,x.answer,x.Q_FFNN_embeds),axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (1078 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
#function to get the start of positive mask to be used while creating the masks
def mask_start(gpt_data):
  return 1024-gpt_data[::-1].index(4600)+1

In [ ]:
#finding the mask start for the train dataset
qa['mask_start']=qa.gpt_data.progress_apply(lambda x: mask_start(x))

In [ ]:
#finding the mask start for the validation dataset
validation_gpt_data['mask_start']=validation_gpt_data.gpt_data.progress_apply(lambda x: mask_start(x))

In [ ]:
#saving train data to disk
qa.to_pickle('./qa_gpt_inbuild.pkl')

In [ ]:
#finding length of the gpt data for train dataset
qa['gpt_lens']=qa.gpt_data.apply(lambda x: len(x))

In [ ]:
#finding length of the gpt data for validation dataset
validation_gpt_data['gpt_lens']=validation_gpt_data.gpt_data.apply(lambda x: len(x))

In [ ]:
#finding number of train datapoints that has sequence length less than 1024
qa[qa.gpt_lens<1024].shape

(19, 7)

In [ ]:
#finding number of validation datapoints that has sequence length less than 1024
validation_gpt_data[validation_gpt_data.gpt_lens<1024].shape

(7, 7)

In [ ]:
#finding train datapoints that has sequence length of 1024
gpt_data_cleaned=qa[qa.gpt_lens==1024]
gpt_data_cleaned.shape

(23783, 7)

In [ ]:
#finding validation datapoints that has sequence length of 1024
validation_gpt_data_cleaned=validation_gpt_data[validation_gpt_data.gpt_lens==1024]
validation_gpt_data_cleaned.shape

(5943, 7)

In [ ]:
#function to create the loss mask given mask start
def return_loss_mask(mask_start):
  return [0]*mask_start+[1]*(1024-mask_start)

In [ ]:
#creating loss mask for train data
gpt_data_cleaned['mask']=gpt_data_cleaned.mask_start.apply(lambda x: return_loss_mask(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#creating loss mask for validation data
validation_gpt_data_cleaned['mask']=validation_gpt_data_cleaned.mask_start.apply(lambda x: return_loss_mask(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#saving train gpt data with loss mask to disk
gpt_data_cleaned.to_pickle('./qa_gpt_inbuild_cleaned.pkl')

In [ ]:
#saving validation gpt data with loss mask to disk
validation_gpt_data_cleaned.to_pickle('./validation_gpt_inbuild_cleaned.pkl')

In [ ]:
#loading gpt train and validation data from disk
gpt_data_cleaned=pd.read_pickle('./qa_gpt_inbuild_cleaned.pkl')
validation_gpt_data_cleaned=pd.read_pickle('./validation_gpt_inbuild_cleaned.pkl')
gpt_data_cleaned.shape,validation_gpt_data_cleaned.shape

((23783, 8), (5943, 8))

In [ ]:
#extracting gpt data for train data
gpt_data=[]
for i in gpt_data_cleaned.gpt_data.values:
  gpt_data.append(i)
gpt_data=np.array(gpt_data)
gpt_data.shape

(23783, 1024)

In [ ]:
#extracting gpt data for validation data
validation_gpt_data=[]
for i in validation_gpt_data_cleaned.gpt_data.values:
  validation_gpt_data.append(i)
validation_gpt_data=np.array(validation_gpt_data)
validation_gpt_data.shape

(5943, 1024)

In [ ]:
#extracting loss mask for train data
loss_mask=[]
for i in gpt_data_cleaned['mask'].values:
  loss_mask.append(i)
loss_mask=np.array(loss_mask)
loss_mask.shape

(23783, 1024)

In [ ]:
#extracting loss mask for validation data
validation_loss_mask=[]
for i in validation_gpt_data_cleaned['mask'].values:
  validation_loss_mask.append(i)
validation_loss_mask=np.array(validation_loss_mask)
validation_loss_mask.shape

(5943, 1024)

In [ ]:
#https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html
#preparing the gpt train dataset
import tensorflow as tf
BATCH_SIZE = 1
BUFFER_SIZE = 20000
gpt_train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'context': gpt_data,

        'label': gpt_data[:,1:],
        'loss_mask':loss_mask[:,:-1]
    },
))


gpt_train_dataset = gpt_train_dataset.shuffle(BUFFER_SIZE)
gpt_train_dataset = gpt_train_dataset.batch(BATCH_SIZE)
gpt_train_dataset = gpt_train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
#https://blog.tensorflow.org/2019/05/transformer-chatbot-tutorial-with-tensorflow-2.html
#preparing the gpt validation dataset
import tensorflow as tf
BATCH_SIZE = 1
BUFFER_SIZE = 20000
validation_gpt_train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'context': validation_gpt_data,

        'label': validation_gpt_data[:,1:],
        'loss_mask':validation_loss_mask[:,:-1]
    },
))

validation_gpt_train_dataset = validation_gpt_train_dataset.shuffle(BUFFER_SIZE)
validation_gpt_train_dataset = validation_gpt_train_dataset.batch(BATCH_SIZE)
validation_gpt_train_dataset = validation_gpt_train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 7.7MB/s 
     |████████████████████████████████| 890kB 34.4MB/s 
     |████████████████████████████████| 3.2MB 54.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=9488e774ad6fa4c69153ed168d21220491407c64d48cafc37e1065c86d2894a9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
#loading pre-trained gpt2 tokenizer and gpt2 model
from transformers import GPT2Tokenizer,TFGPT2LMHeadModel
tokenizer=GPT2Tokenizer.from_pretrained("gpt2")
tf_gpt2_model=TFGPT2LMHeadModel.from_pretrained("gpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
#displaying sample gpt training data
next(iter(gpt_train_dataset))

({'context': <tf.Tensor: shape=(1, 1024), dtype=int64, numpy=array([[ 326, 1312,  714, ...,  290, 1332, 2482]])>,
  'label': <tf.Tensor: shape=(1, 1023), dtype=int64, numpy=array([[1312,  714,  307, ...,  290, 1332, 2482]])>,
  'loss_mask': <tf.Tensor: shape=(1, 1023), dtype=int64, numpy=array([[0, 0, 0, ..., 1, 1, 1]])>},)

In [ ]:
#training the gpt2 model
learning_rate=0.0001
optim=tf.optimizers.Adam(learning_rate)
j=0
loss_cum=0
epoch_loss=0
stop_training=False
epochs=10
for epoch in range(epochs):
  print('epoch',epoch)
  for i in tqdm(gpt_train_dataset):
    with tf.GradientTape() as tape:
      context=i[0]['context']
      loss_mask=tf.cast(i[0]['loss_mask'],tf.float32)
      output=tf_gpt2_model(context,training=True)['logits']
      loss_value = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=np.array(i[0]['label']), logits=output[ :,:-1])*loss_mask)
      j=j+1
      loss_cum+=loss_value
      epoch_loss+=loss_value/100
      if j%50==0:
        print(loss_cum/50)
        grads=tape.gradient(loss_cum/50,tf_gpt2_model.trainable_variables)
        optim.apply_gradients(zip(grads,tf_gpt2_model.trainable_variables))
        loss_cum=0
      if tf.math.is_nan(loss_value):
        print('loss became nan')
        stop_training=True
        break
      if j%10000==0:
        tf_gpt2_model.save_pretrained("tf_gpt2_model_2_10"+str(epoch)+'_'+str(j))
  print('epoch_loss by 100',epoch_loss/237)
  epoch_loss=0
  if stop_training:
    break
  tf_gpt2_model.save_pretrained("tf_gpt2_model_2_10"+str(epoch))
      
    

epoch 0


tf.Tensor(364.4527, shape=(), dtype=float32)
tf.Tensor(370.41684, shape=(), dtype=float32)
tf.Tensor(351.39758, shape=(), dtype=float32)
tf.Tensor(388.43634, shape=(), dtype=float32)
tf.Tensor(353.1692, shape=(), dtype=float32)
tf.Tensor(492.08533, shape=(), dtype=float32)
tf.Tensor(390.4793, shape=(), dtype=float32)
tf.Tensor(351.06656, shape=(), dtype=float32)
tf.Tensor(483.38742, shape=(), dtype=float32)
tf.Tensor(408.05316, shape=(), dtype=float32)
tf.Tensor(398.0508, shape=(), dtype=float32)
tf.Tensor(455.85046, shape=(), dtype=float32)
tf.Tensor(453.43805, shape=(), dtype=float32)
tf.Tensor(282.57803, shape=(), dtype=float32)
tf.Tensor(416.11826, shape=(), dtype=float32)
tf.Tensor(411.5636, shape=(), dtype=float32)
tf.Tensor(410.40692, shape=(), dtype=float32)
tf.Tensor(338.4495, shape=(), dtype=float32)
tf.Tensor(397.2391, shape=(), dtype=float32)
tf.Tensor(388.54718, shape=(), dtype=float32)
tf.Tensor(494.09085, shape=(), dtype=float32)
tf.Tensor(496.60992, shape=(), dtype=floa

tf.Tensor(418.09238, shape=(), dtype=float32)
tf.Tensor(457.12225, shape=(), dtype=float32)
tf.Tensor(431.54153, shape=(), dtype=float32)
tf.Tensor(332.30743, shape=(), dtype=float32)
tf.Tensor(237.00127, shape=(), dtype=float32)
tf.Tensor(574.28925, shape=(), dtype=float32)
tf.Tensor(356.4869, shape=(), dtype=float32)
tf.Tensor(351.42197, shape=(), dtype=float32)
tf.Tensor(325.27115, shape=(), dtype=float32)
tf.Tensor(463.2143, shape=(), dtype=float32)
tf.Tensor(457.21545, shape=(), dtype=float32)
tf.Tensor(314.66214, shape=(), dtype=float32)
tf.Tensor(285.07645, shape=(), dtype=float32)
tf.Tensor(390.65784, shape=(), dtype=float32)
tf.Tensor(307.58102, shape=(), dtype=float32)
tf.Tensor(312.74942, shape=(), dtype=float32)
tf.Tensor(318.08356, shape=(), dtype=float32)
tf.Tensor(375.7545, shape=(), dtype=float32)
tf.Tensor(357.14465, shape=(), dtype=float32)
tf.Tensor(374.43567, shape=(), dtype=float32)
tf.Tensor(248.84195, shape=(), dtype=float32)
tf.Tensor(397.43375, shape=(), dtype=

tf.Tensor(391.69446, shape=(), dtype=float32)
tf.Tensor(301.40253, shape=(), dtype=float32)
tf.Tensor(311.38867, shape=(), dtype=float32)
tf.Tensor(386.22372, shape=(), dtype=float32)
tf.Tensor(296.69232, shape=(), dtype=float32)
tf.Tensor(320.2208, shape=(), dtype=float32)
tf.Tensor(360.03043, shape=(), dtype=float32)
tf.Tensor(313.58087, shape=(), dtype=float32)
tf.Tensor(505.28784, shape=(), dtype=float32)
tf.Tensor(383.17957, shape=(), dtype=float32)
tf.Tensor(463.07858, shape=(), dtype=float32)
tf.Tensor(408.0493, shape=(), dtype=float32)
tf.Tensor(353.21945, shape=(), dtype=float32)
tf.Tensor(337.01578, shape=(), dtype=float32)
tf.Tensor(416.4724, shape=(), dtype=float32)
tf.Tensor(360.25137, shape=(), dtype=float32)
tf.Tensor(407.17438, shape=(), dtype=float32)
tf.Tensor(295.73465, shape=(), dtype=float32)
tf.Tensor(335.32672, shape=(), dtype=float32)
tf.Tensor(300.17108, shape=(), dtype=float32)
tf.Tensor(354.70712, shape=(), dtype=float32)
tf.Tensor(308.57318, shape=(), dtype=

tf.Tensor(371.44473, shape=(), dtype=float32)
tf.Tensor(342.2949, shape=(), dtype=float32)
tf.Tensor(363.94864, shape=(), dtype=float32)
tf.Tensor(389.8552, shape=(), dtype=float32)
tf.Tensor(349.5277, shape=(), dtype=float32)
tf.Tensor(360.65057, shape=(), dtype=float32)
tf.Tensor(358.9205, shape=(), dtype=float32)
tf.Tensor(428.234, shape=(), dtype=float32)
tf.Tensor(369.20038, shape=(), dtype=float32)
tf.Tensor(397.87122, shape=(), dtype=float32)
tf.Tensor(340.41592, shape=(), dtype=float32)
tf.Tensor(359.37332, shape=(), dtype=float32)
tf.Tensor(354.04358, shape=(), dtype=float32)
tf.Tensor(283.43625, shape=(), dtype=float32)
tf.Tensor(320.52887, shape=(), dtype=float32)
tf.Tensor(354.40265, shape=(), dtype=float32)
tf.Tensor(256.52225, shape=(), dtype=float32)
tf.Tensor(415.28497, shape=(), dtype=float32)
tf.Tensor(390.62686, shape=(), dtype=float32)
tf.Tensor(267.5209, shape=(), dtype=float32)
tf.Tensor(399.92087, shape=(), dtype=float32)
tf.Tensor(392.6946, shape=(), dtype=float

tf.Tensor(335.77548, shape=(), dtype=float32)
tf.Tensor(412.41193, shape=(), dtype=float32)
tf.Tensor(289.7776, shape=(), dtype=float32)
tf.Tensor(339.27206, shape=(), dtype=float32)
tf.Tensor(372.82547, shape=(), dtype=float32)
tf.Tensor(340.60928, shape=(), dtype=float32)
tf.Tensor(334.7916, shape=(), dtype=float32)
tf.Tensor(393.78934, shape=(), dtype=float32)
tf.Tensor(348.67673, shape=(), dtype=float32)
tf.Tensor(348.81598, shape=(), dtype=float32)
tf.Tensor(372.4725, shape=(), dtype=float32)
tf.Tensor(294.45792, shape=(), dtype=float32)
tf.Tensor(349.25534, shape=(), dtype=float32)
tf.Tensor(406.88617, shape=(), dtype=float32)
tf.Tensor(365.7967, shape=(), dtype=float32)
tf.Tensor(373.11914, shape=(), dtype=float32)
tf.Tensor(374.54648, shape=(), dtype=float32)
tf.Tensor(292.98203, shape=(), dtype=float32)
tf.Tensor(348.93552, shape=(), dtype=float32)
tf.Tensor(393.13885, shape=(), dtype=float32)
tf.Tensor(436.86682, shape=(), dtype=float32)
tf.Tensor(368.63083, shape=(), dtype=f

tf.Tensor(258.32166, shape=(), dtype=float32)
tf.Tensor(450.4697, shape=(), dtype=float32)
tf.Tensor(367.4019, shape=(), dtype=float32)
tf.Tensor(382.78555, shape=(), dtype=float32)
tf.Tensor(284.64178, shape=(), dtype=float32)
tf.Tensor(423.14117, shape=(), dtype=float32)
tf.Tensor(324.2942, shape=(), dtype=float32)
tf.Tensor(235.01459, shape=(), dtype=float32)
tf.Tensor(374.83038, shape=(), dtype=float32)
tf.Tensor(304.6962, shape=(), dtype=float32)
tf.Tensor(274.21497, shape=(), dtype=float32)
tf.Tensor(412.9756, shape=(), dtype=float32)
tf.Tensor(301.85733, shape=(), dtype=float32)
tf.Tensor(286.0098, shape=(), dtype=float32)
tf.Tensor(301.80267, shape=(), dtype=float32)
tf.Tensor(367.15738, shape=(), dtype=float32)
tf.Tensor(292.96896, shape=(), dtype=float32)
tf.Tensor(323.6565, shape=(), dtype=float32)
tf.Tensor(278.91724, shape=(), dtype=float32)
tf.Tensor(453.87668, shape=(), dtype=float32)
tf.Tensor(372.9174, shape=(), dtype=float32)
tf.Tensor(477.38242, shape=(), dtype=float

In [ ]:
#loading the gpt2 model from the above saved checkpoint
tf_gpt2_model=TFGPT2LMHeadModel.from_pretrained("./tf_gpt2_model_2_105_130000")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./tf_gpt2_model_2_105_130000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
#training the gpt2 model
learning_rate=0.0001
optim=tf.optimizers.Adam(learning_rate)
j=0
loss_cum=0
epoch_loss=0
stop_training=False
epochs=10
for epoch in range(epochs):
  print('epoch',epoch)
  for i in tqdm(gpt_train_dataset):
    with tf.GradientTape() as tape:
      context=i[0]['context']
      loss_mask=tf.cast(i[0]['loss_mask'],tf.float32)
      output=tf_gpt2_model(context,training=True)['logits']
      loss_value = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=np.array(i[0]['label']), logits=output[ :,:-1])*loss_mask)
      j=j+1
      loss_cum+=loss_value
      epoch_loss+=loss_value/100
      if j%50==0:
        print(loss_cum/50)
        grads=tape.gradient(loss_cum/50,tf_gpt2_model.trainable_variables)
        optim.apply_gradients(zip(grads,tf_gpt2_model.trainable_variables))
        loss_cum=0
      if tf.math.is_nan(loss_value):
        print('loss became nan')
        stop_training=True
        break
      if j%10000==0:
        tf_gpt2_model.save_pretrained("tf_gpt2_model_2_11"+str(epoch)+'_'+str(j))
  print('epoch_loss by 100',epoch_loss/237)
  epoch_loss=0
  if stop_training:
    break
  tf_gpt2_model.save_pretrained("tf_gpt2_model_2_11"+str(epoch))
      
    

epoch 0


tf.Tensor(292.36526, shape=(), dtype=float32)
tf.Tensor(333.8749, shape=(), dtype=float32)
tf.Tensor(254.67285, shape=(), dtype=float32)
tf.Tensor(297.27075, shape=(), dtype=float32)
tf.Tensor(426.60907, shape=(), dtype=float32)
tf.Tensor(291.14005, shape=(), dtype=float32)
tf.Tensor(382.55676, shape=(), dtype=float32)
tf.Tensor(340.3334, shape=(), dtype=float32)
tf.Tensor(291.28552, shape=(), dtype=float32)
tf.Tensor(333.2575, shape=(), dtype=float32)
tf.Tensor(248.03618, shape=(), dtype=float32)
tf.Tensor(345.12274, shape=(), dtype=float32)
tf.Tensor(333.4293, shape=(), dtype=float32)
tf.Tensor(382.98578, shape=(), dtype=float32)
tf.Tensor(467.57532, shape=(), dtype=float32)
tf.Tensor(253.18805, shape=(), dtype=float32)
tf.Tensor(446.7239, shape=(), dtype=float32)
tf.Tensor(324.61765, shape=(), dtype=float32)
tf.Tensor(297.08206, shape=(), dtype=float32)
tf.Tensor(328.3179, shape=(), dtype=float32)
tf.Tensor(367.7425, shape=(), dtype=float32)
tf.Tensor(364.3258, shape=(), dtype=float

tf.Tensor(376.2625, shape=(), dtype=float32)
tf.Tensor(261.71606, shape=(), dtype=float32)
tf.Tensor(436.11624, shape=(), dtype=float32)
tf.Tensor(374.94186, shape=(), dtype=float32)
tf.Tensor(303.3938, shape=(), dtype=float32)
tf.Tensor(324.9739, shape=(), dtype=float32)
tf.Tensor(332.53555, shape=(), dtype=float32)
tf.Tensor(330.80988, shape=(), dtype=float32)
tf.Tensor(364.27927, shape=(), dtype=float32)
tf.Tensor(417.3026, shape=(), dtype=float32)
tf.Tensor(368.0514, shape=(), dtype=float32)
tf.Tensor(463.49905, shape=(), dtype=float32)
tf.Tensor(394.934, shape=(), dtype=float32)
tf.Tensor(389.34387, shape=(), dtype=float32)
tf.Tensor(301.35272, shape=(), dtype=float32)
tf.Tensor(361.16104, shape=(), dtype=float32)
tf.Tensor(356.6364, shape=(), dtype=float32)
tf.Tensor(274.9276, shape=(), dtype=float32)
tf.Tensor(344.66083, shape=(), dtype=float32)
tf.Tensor(288.2619, shape=(), dtype=float32)
tf.Tensor(356.36465, shape=(), dtype=float32)
tf.Tensor(370.88962, shape=(), dtype=float32

tf.Tensor(329.75568, shape=(), dtype=float32)
tf.Tensor(347.72424, shape=(), dtype=float32)
tf.Tensor(329.85486, shape=(), dtype=float32)
tf.Tensor(273.2921, shape=(), dtype=float32)
tf.Tensor(403.0063, shape=(), dtype=float32)
tf.Tensor(398.3734, shape=(), dtype=float32)
tf.Tensor(368.03455, shape=(), dtype=float32)
tf.Tensor(269.6875, shape=(), dtype=float32)
tf.Tensor(420.69122, shape=(), dtype=float32)
tf.Tensor(263.7334, shape=(), dtype=float32)
tf.Tensor(280.10248, shape=(), dtype=float32)
tf.Tensor(247.07805, shape=(), dtype=float32)
tf.Tensor(315.027, shape=(), dtype=float32)
tf.Tensor(317.95526, shape=(), dtype=float32)
tf.Tensor(288.54935, shape=(), dtype=float32)
tf.Tensor(338.89175, shape=(), dtype=float32)
tf.Tensor(296.6603, shape=(), dtype=float32)
tf.Tensor(357.23828, shape=(), dtype=float32)
tf.Tensor(302.69653, shape=(), dtype=float32)
tf.Tensor(264.52774, shape=(), dtype=float32)
tf.Tensor(247.10063, shape=(), dtype=float32)
tf.Tensor(398.04465, shape=(), dtype=float

tf.Tensor(383.25784, shape=(), dtype=float32)
tf.Tensor(255.48932, shape=(), dtype=float32)
tf.Tensor(333.7997, shape=(), dtype=float32)
tf.Tensor(396.4166, shape=(), dtype=float32)
tf.Tensor(331.56485, shape=(), dtype=float32)
tf.Tensor(446.99164, shape=(), dtype=float32)
tf.Tensor(311.78928, shape=(), dtype=float32)
tf.Tensor(323.14215, shape=(), dtype=float32)
tf.Tensor(448.80682, shape=(), dtype=float32)
tf.Tensor(343.84534, shape=(), dtype=float32)
tf.Tensor(334.0533, shape=(), dtype=float32)
tf.Tensor(316.87668, shape=(), dtype=float32)
tf.Tensor(285.10764, shape=(), dtype=float32)
tf.Tensor(350.9965, shape=(), dtype=float32)
tf.Tensor(390.06125, shape=(), dtype=float32)
tf.Tensor(331.81985, shape=(), dtype=float32)
tf.Tensor(262.83078, shape=(), dtype=float32)
tf.Tensor(383.64374, shape=(), dtype=float32)
tf.Tensor(380.20004, shape=(), dtype=float32)
tf.Tensor(440.90894, shape=(), dtype=float32)
tf.Tensor(337.65848, shape=(), dtype=float32)
tf.Tensor(366.43512, shape=(), dtype=f

tf.Tensor(408.02582, shape=(), dtype=float32)
tf.Tensor(354.7493, shape=(), dtype=float32)
tf.Tensor(352.95035, shape=(), dtype=float32)
tf.Tensor(348.47574, shape=(), dtype=float32)
tf.Tensor(400.20496, shape=(), dtype=float32)
tf.Tensor(323.05475, shape=(), dtype=float32)
tf.Tensor(313.8267, shape=(), dtype=float32)
tf.Tensor(301.70032, shape=(), dtype=float32)
tf.Tensor(348.7218, shape=(), dtype=float32)
tf.Tensor(292.7201, shape=(), dtype=float32)
tf.Tensor(415.3608, shape=(), dtype=float32)
tf.Tensor(319.2008, shape=(), dtype=float32)
tf.Tensor(286.9254, shape=(), dtype=float32)
tf.Tensor(341.8368, shape=(), dtype=float32)
tf.Tensor(297.88336, shape=(), dtype=float32)
tf.Tensor(315.31155, shape=(), dtype=float32)
tf.Tensor(427.58386, shape=(), dtype=float32)
tf.Tensor(350.24634, shape=(), dtype=float32)
tf.Tensor(305.58963, shape=(), dtype=float32)
tf.Tensor(280.98514, shape=(), dtype=float32)
tf.Tensor(403.31073, shape=(), dtype=float32)
tf.Tensor(387.412, shape=(), dtype=float32

In [ ]:
#loading the model from the saved checkpoint above
tf_gpt2_model=TFGPT2LMHeadModel.from_pretrained("./tf_gpt2_model_2_115_120000")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./tf_gpt2_model_2_105_130000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
#function to return sample origingal and generated answers for the train dataset
def return_sample_output_predicted():
  sample_data=next(iter(gpt_train_dataset))
  v=sample_data[0]['context'].numpy()[0]
  mask_start = len(v) - list(v[::-1]).index(4600) + 1
  input=v[:mask_start+1]
  next_step_input=[]
  input1=list(input)+next_step_input
  final_output=[]
  for i in range(0,1024-len(input)):
    input1=list(input1)+next_step_input
    output=tf_gpt2_model([np.array(input1)])['logits']
    output1=np.argmax(output,axis=1)
    next_step_input=[output1[-1]]#appending last output to input
    next_step_char=tokenizer.decode(next_step_input)
    final_output.append(next_step_char)
  expected_output=v[mask_start+1:]
  return v,tokenizer.decode(expected_output),' '.join(final_output)





In [ ]:
#displaying sample original context, expected answer and generated answer for train datset
for i in range(5):
  v,exp,fin=return_sample_output_predicted()
  print(tokenizer.decode(v))
  print(exp)
  print(fin)
  print('------------------')

 can tell you it took a long time for the injury to heal because we use our hands for everything but with time i am able to still use my hand and the pain is gone splint the finger and keep it on for a while `QUESTION: what causes pain when you lower your arm after raising it  i injured my arm lifting a box of books had an xray which showed only some arthritis have had some problem with pain in my shoulder when sleeping on it in the past it is my right shoulder `ANSWER: i have had surgery on my rotator cuff because of natural wear i had a problem that if i raised and then lowered my arm the shoulder hurt when i lowered my arm it turned out to be shoulder rotator cartilage problem the pain disappeared after a few moments after i lowered my arm `QUESTION: what are the different surgical options for treatment of arthritis of the hand and wrist `ANSWER: treatmentsurgery for painful arthritic fingers and how to prevent further deterioration `QUESTION: arthritis on its own or arthritis as a 

In [ ]:
#function to return sample origingal and generated answers for the validation dataset
def return_validation_sample_output_predicted():
  sample_data=next(iter(validation_gpt_train_dataset))
  v=sample_data[0]['context'].numpy()[0]
  mask_start = len(v) - list(v[::-1]).index(4600) + 1
  input=v[:mask_start+1]
  next_step_input=[]
  input1=list(input)+next_step_input
  final_output=[]
  token_output=[]
  for i in range(0,1024-len(input)):
    input1=list(input1)+next_step_input
    output=tf_gpt2_model([np.array(input1)])['logits']
    output1=np.argmax(output,axis=1)
    next_step_input=[output1[-1]]#appending last output to input
    next_step_char=tokenizer.decode(next_step_input)
    token_output.append(next_step_input[0])
    final_output.append(next_step_char)
  expected_output=v[mask_start+1:]
  return v,tokenizer.decode(expected_output),tokenizer.decode(token_output)





In [ ]:
#displaying sample original context, expected answer and generated answer for validation datset
for i in range(5):
  v,exp,fin=return_validation_sample_output_predicted()
  print(tokenizer.decode(v))
  print(exp)
  print(fin)
  print('------------------')

 to healthcare gov and begin the process `QUESTION: i have medicare now   my husband has none   how does that work with the new health insurance policies  will my medicare change what services and who the doctors it covers how about medicaid `ANSWER: the marketplaces are not for people currently covered by or newly eligible for medicare `QUESTION: what is the official website for obamacare `ANSWER: healthcare gov is the official federal website `QUESTION: has qualifying for medicare or medicaid become easier for those aged 55 65 `ANSWER: you cannot qualify for medicare before the age of 65 `QUESTION: why does health insurance matter for those who are young and healthy `ANSWER: although it is true that young people are healthier on average as a group they do face health problems too about 16 deal with chronic illnesses that require them to seek care on a regular basis and young adults between the ages 19 and 29 visit the emergency room more than any other group under the age of 75 mostl

**Conclusion**

From the above sample generated answes we can observe that the gpt2 model generates decent medical answers